In [1]:
!pip install tensorflow==2.1.0

import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 421.8MB 19kB/s 
     |████████████████████████████████| 3.9MB 51.9MB/s 
     |████████████████████████████████| 450kB 54.5MB/s 
     |████████████████████████████████| 51kB 9.0MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=91c982e26b76a9bf9b8991aee6018a13ab909b7e6703a0661d0277a9961c8d34
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tens

In [2]:
!gdown --id "1rtLYlLQ9QIMEb5v2eH1D9mu7FpgOqzYq"

Downloading...
From: https://drive.google.com/uc?id=1rtLYlLQ9QIMEb5v2eH1D9mu7FpgOqzYq
To: /content/dataset_lab3_SRCIM_400.zip
13.5MB [00:00, 119MB/s]


In [3]:
import os
import zipfile

local_zip = "/content/dataset_lab3_SRCIM_400.zip"
zip_ref = zipfile.ZipFile(local_zip, "r")
zip_ref.extractall('/tmp')

zip_ref.close()

In [4]:
data_path = "/tmp/dataset"

IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_path)

datagen_kwargs = dict(
    rescale=1./255,
    validation_split=0.20
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

Found 320 images belonging to 2 classes.
Found 80 images belonging to 2 classes.


In [5]:
#Initiate Sequential Model
model = tf.keras.models.Sequential()

#First Convolutional Layer
model.add(
  tf.keras.layers.Conv2D(
    filters=32,
    kernel_size=(3, 3),
    input_shape=(224, 224, 3),
    activation='relu'
  )
)

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Second Convolution Layer
model.add(
  tf.keras.layers.Conv2D(
    filters=64,
    kernel_size=(3, 3),
    activation='relu'
  )
)

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Third Convolutional Layer
model.add(
  tf.keras.layers.Conv2D(
    filters=128,
    kernel_size=(3, 3),
    activation='relu'
  )
)

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Fourth Convolutional Layer
model.add(
  tf.keras.layers.Conv2D(
    filters=256,
    kernel_size=(3, 3),
    activation='relu'
  )
)

model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))

#Flatten the fooled feature map
model.add(tf.keras.layers.Flatten())

#Add the fully connected layer with a ReLU
model.add(tf.keras.layers.Dense(256))
model.add(tf.keras.layers.Activation('relu'))

#Add output layer
model.add(tf.keras.layers.Dense(train_generator.num_classes, activation="softmax"))

#Build model and print
model.build([None, 224, 224, 3])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 26, 26, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 24, 24, 256)       2

In [6]:
#Compile the model
optimizer = tf.keras.optimizers.Adam()

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=['acc']
)

In [7]:
#Model training and validation
import numpy as np

callbacks = [tf.keras.callbacks.EarlyStopping(patience=5)]

steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)

hist = model.fit(
    train_generator,
    epochs=50,
    verbose=1,
    #callbacks=callbacks,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=val_steps_per_epoch
).history

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 10.0 steps, validate for 3.0 steps
Epoch 1/50
10/10 [==============================] - 9s 926ms/step - loss: 0.9785 - acc: 0.4938 - val_loss: 0.7003 - val_acc: 0.5000
Epoch 2/50
10/10 [==============================] - 2s 194ms/step - loss: 0.6954 - acc: 0.5000 - val_loss: 0.6934 - val_acc: 0.5000
Epoch 3/50
10/10 [==============================] - 2s 193ms/step - loss: 0.6931 - acc: 0.5000 - val_loss: 0.6932 - val_acc: 0.5000
Epoch 4/50
10/10 [==============================] - 2s 193ms/step - loss: 0.6932 - acc: 0.5156 - val_loss: 0.6931 - val_acc: 0.4875
Epoch 5/50
10/10 [==============================] - 2s 195ms/step - loss: 0.6932 - acc: 0.5156 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 6/50
10/10 [==============================] - 2s 194ms/step - loss: 0.6932 - acc: 0.4750 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 7/50
10/10 [==============================] - 2s 196ms/step - loss: 0.6931 - acc: 0.5219 - val_loss: 0.6932

In [ ]:
#Vizualize the training history in termos of accuracy and loss
import matplotlib.pyplot as plt

plt.figure()
plt.ylabel("Loss (training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0, 2])
plt.plot(hist["loss"])
plt.plot(hist["val_loss"])
plt.figure()
plt.ylabel("Accuracy(training and validation)")
plt.xlabel("Training Steps")
plt.ylim([0.6, 1])
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])

In [8]:
#Check final loss and accuracy results
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final Loss: {:.2f}".format(final_loss))
print("Final Accuracy: {:2f}%".format(final_accuracy * 100))

  ...
    to  
  ['...']
3/3 [==============================] - 0s 117ms/step - loss: 0.1026 - acc: 0.9750
Final Loss: 0.10
Final Accuracy: 97.500002%


In [10]:
#Save trained model
model.save("srcim_model.h5")
print("Model saved")

Model saved
